In [16]:
import os
import nest_asyncio
import pandas as pd
from pgpt_python.client import PrivateGPTApi
from typing import List, Dict
from collections import Counter
from llama_index.core.evaluation import RelevancyEvaluator, CorrectnessEvaluator, FaithfulnessEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core.llama_dataset import LabelledRagDataset, LabelledRagDataExample
import tiktoken

In [3]:
client = PrivateGPTApi(base_url='http://localhost:8001')
nest_asyncio.apply()

In [3]:
os.makedirs('data/', exist_ok=True)
# Load data
reader = SimpleDirectoryReader('./data/')
documents = reader.load_data()

In [6]:
# Create vector index
vector_index = VectorStoreIndex.from_documents(documents)
query_engine = vector_index.as_query_engine()

/Users/qingqingdong/.pyenv/versions/private_env/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.30566223168627227 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
/Users/qingqingdong/.pyenv/versions/private_env/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryR

In [4]:
import json

# Load the dictionary from the local JSON file
with open('questions_answers.json', 'r') as f:
    questions_answers = json.load(f)

# Convert the dictionary to a list of questions and answers using a list comprehension
questions_answers_list = [(q, questions_answers[q]) for q in questions_answers]

# Extract the questions and answers from the list
eval_questions, answers = zip(*questions_answers_list)

In [73]:
# Generate reference answers for evaluation using PrivateGPT
system_prompt_context = '''
    Instruction: You are an IT security expert for a famous German company called Hydac.
    Given the question and related context, generate a short, clear, and professional response in German.
    You can refer to the example give below -----
    Question: Inwieweit werden alle Mitarbeiter zur Einhaltung der Informationssicherheit verpflichtet?
    Context: 2.1.2 Inwieweit werden alle Mitarbeiter zur Einhaltung der Informationssicherheit verpflichtet?
    Detaillierte Sachverhaltsdarstellung (inkl. Beurteilungsverfahren) Betrachtete Dokumente/Nachweise/Prüfungshandlung:
    Datengeheimnis Arbeitsvertrag,  IT-SRL, Vorlagen  Datenschutz Statement Hydac: Im Arbeitsvertrag wird auf die Einhaltung
    von Verfahrensanweisungen und im Unternehmen geltenden Richtlinien sowie Geheimhaltung verpflichtet. Mit Abschluss des Arbeitsvertrages 
    erhält der Mitarbeiter eine Erklärung zum Datengeheimnis, die er mit dem Unterzeichnen einwilligt. Weitere Verpflichtungen zur Geheimhaltung 
    können je nach Berufsgruppe bestehen und werden durch den Zentralbereich Datenschutz ausgegeben. Die IT-SRL ist eine Verfahrensanweisung 
    und somit für alle Mitarbeiter bindend. Mitarbeiter werden zur Geheimhaltung und auf das Regelwerk zur Informationssicherheit verpflichtet. 
    Feststellung Auf Basis der Beobachtungen wurde keine Abweichung festgestellt. 
    Answer: Im Arbeitsvertrag wird auf die Einhaltung von Verfahrensanweisungen und im Unternehmen geltenden Richtlinien sowie Geheimhaltung 
    verpflichtet. Mit Abschluss des Arbeitsvertrages erhält der Mitarbeiter eine Erklärung zum Datengeheimnis, die er mit dem Unterzeichnen einwilligt. 
    Weitere Verpflichtungen zur Geheimhaltung können je nach Berufsgruppe bestehen und werden durch den Zentralbereich Datenschutz ausgegeben. 
    Die IT-SRL ist eine Verfahrensanweisung und somit für alle Mitarbeiter bindend Mitarbeiter werden zur Geheimhaltung und auf das Regelwerk zur Informationssicherheit verpflichtet..
'''

system_prompt_without_context = '''
    Instruction: You are an IT security expert for a famous German company called Hydac.
    Given the question, generate a short, clear, and professional response in German.
    You can refer to the example give below -----
    Question: Inwieweit werden alle Mitarbeiter zur Einhaltung der Informationssicherheit verpflichtet?
    Answer: Im Arbeitsvertrag wird auf die Einhaltung von Verfahrensanweisungen und im Unternehmen geltenden Richtlinien sowie Geheimhaltung 
    verpflichtet. Mit Abschluss des Arbeitsvertrages erhält der Mitarbeiter eine Erklärung zum Datengeheimnis, die er mit dem Unterzeichnen einwilligt. 
    Weitere Verpflichtungen zur Geheimhaltung können je nach Berufsgruppe bestehen und werden durch den Zentralbereich Datenschutz ausgegeben. 
    Die IT-SRL ist eine Verfahrensanweisung und somit für alle Mitarbeiter bindend Mitarbeiter werden zur Geheimhaltung und auf das Regelwerk zur Informationssicherheit verpflichtet..
'''

In [15]:
reference_answers = []
for question in eval_questions:
    response = client.contextual_completions.prompt_completion(
        prompt=question,
        use_context=True,
        include_sources=True,
        system_prompt=system_prompt_context
    ).choices[0]
    reference_answers.append({
        "content": response.message.content,
        "sources": [c.text for c in response.sources]
    })
    # Write the updated list to the file
    with open('answers.json', 'w') as f:
        json.dump(reference_answers, f)

In [ ]:
reference_answers = []
for question in eval_questions:
    response = client.contextual_completions.prompt_completion(
        prompt=question,
        system_prompt=system_prompt_without_context
    ).choices[0]
    reference_answers.append({
        "content": response.message.content,
    })
    # Write the updated list to the file
    with open('llama3_answers_zeroshot.json', 'w') as f:
        json.dump(reference_answers, f)

In [77]:
# instantiate the gpt-4 judges
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    AnswerRelevancyEvaluator,
    ContextRelevancyEvaluator,
)


os.environ["OPENAI_API_KEY"] = ""

judges = {}

judges["answer_relevancy"] = AnswerRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-4"),
)

# judges["context_relevancy"] = ContextRelevancyEvaluator(
#     llm=OpenAI(temperature=0, model="gpt-4"),
# )

judges["correctness"] = CorrectnessEvaluator(
    llm=OpenAI(temperature=0, model="gpt-4"),
)

# judges["faithfulness"] = FaithfulnessEvaluator(
#     llm=OpenAI(temperature=0, model="gpt-4"),
# )

In [75]:
with open('llama3_answers_zeroshot.json', 'r') as f:
    reference_answers = json.load(f)

In [79]:
eval_tasks = []
for i in range(len(eval_questions)):
    eval_tasks.append(
        judges["answer_relevancy"].aevaluate(
            query=eval_questions[i],
            response=reference_answers[i]['content'],
            sleep_time_in_seconds=1.0,
        )
    )
    eval_tasks.append(
        judges["correctness"].aevaluate(
            query=eval_questions[i],
            response=reference_answers[i]['content'],
            reference=answers[i],
            sleep_time_in_seconds=1.0,
        )
    )
    # eval_tasks.append(
    #     judges["context_relevancy"].aevaluate(
    #         query=eval_questions[i],
    #         contexts=reference_answers[i]['sources'],
    #         sleep_time_in_seconds=1.0,
    #     )
    # )
    # eval_tasks.append(
    #     judges["faithfulness"].aevaluate(
    #         response=reference_answers[i]['content'],
    #         contexts=reference_answers[i]['sources'],
    #         sleep_time_in_seconds=1.0,
    # )
    # )

In [80]:
from tqdm.asyncio import tqdm_asyncio
llama3_zeroshot_eval_results = await tqdm_asyncio.gather(*eval_tasks)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:09<00:00,  8.48it/s]


In [81]:
# phi3_evals = {
#     "answer_relevancy": eval_results[::4],
#      "correctness":eval_results[1::4],
#     "context_relevancy": eval_results[2::4],
#      "faithfulness": eval_results[3::4],
# }
# mistral_evals = {
#     "answer_relevancy": mistral_eval_results[::4],
#      "correctness":mistral_eval_results[1::4],
#     "context_relevancy": mistral_eval_results[2::4],
#      "faithfulness": mistral_eval_results[3::4],
# }
# llama3_evals = {
#     "answer_relevancy": llama3_eval_results[::4],
#      "correctness":llama3_eval_results[1::4],
#     "context_relevancy": llama3_eval_results[2::4],
#      "faithfulness": llama3_eval_results[3::4],
# }
# gemma_evals = {
#     "answer_relevancy": gemma_eval_results[::4],
#      "correctness":gemma_eval_results[1::4],
#     "context_relevancy": gemma_eval_results[2::4],
#      "faithfulness": gemma_eval_results[3::4],
# }
llama3_zero_shot_evals = {
    "answer_relevancy": llama3_zeroshot_eval_results[::2],
     "correctness":llama3_zeroshot_eval_results[1::2],
}

In [85]:
from llama_index.core.evaluation.notebook_utils import get_eval_results_df
import pandas as pd

deep_dfs = {}
mean_dfs = {}
for metric in phi3_evals.keys():
    deep_df, mean_df = get_eval_results_df(
        names=["baseline"] * len(phi3_evals[metric]),
        results_arr=phi3_evals[metric],
        metric=metric,
    )
    deep_dfs[metric] = deep_df
    mean_dfs[metric] = mean_df

In [86]:
# phi3
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.963415
mean_correctness_score,4.097561
mean_context_relevancy_score,0.798780
mean_faithfulness_score,1.000000


In [57]:
# mistral
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.963415
mean_correctness_score,4.182927
mean_context_relevancy_score,0.832317
mean_faithfulness_score,1.000000


In [66]:
# llama3
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.963415
mean_correctness_score,4.182927
mean_context_relevancy_score,0.844512
mean_faithfulness_score,1.000000


In [72]:
# gemma
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.951220
mean_correctness_score,4.085366
mean_context_relevancy_score,0.814024
mean_faithfulness_score,1.000000


In [83]:
# llama3_zero_shot
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.951220
mean_correctness_score,3.914634


In [113]:
js = {}
for i in llama3_zero_shot_evals:
        js[i] = [j.json() for j in llama3_zero_shot_evals[i]]
with open('llama3_zero_shot_evals.json', 'w') as f:
        json.dump(js, f)

In [143]:
def process_evals(evals,metrics = ["correctness",'answer_relevancy', 'context_relevancy',"faithfulness"]):
    deep_dfs = {}
    mean_dfs = {}
    for metric in metrics:
        deep_df, mean_df = get_eval_results_df(
            names=["baseline"] * len(evals[metric]),
            results_arr=evals[metric],
            metric=metric,
        )
        deep_dfs[metric] = deep_df
        mean_dfs[metric] = mean_df
    mean_scores_df = pd.concat(
        [mdf.reset_index() for _, mdf in mean_dfs.items()],
        axis=0,
        ignore_index=True,
    )
    mean_scores_df = mean_scores_df.set_index("index")
    mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
    return mean_scores_df

# Process each model's evaluation results


llama3_df = process_evals(llama3_evals)
mistral_df = process_evals(mistral_evals)
phi3_df = process_evals(phi3_evals)
gemma_df = process_evals(gemma_evals)
llama3_zero_shot_df = process_evals(llama3_zero_shot_evals,metrics = ["correctness",'answer_relevancy'])

# Add a column for model names

llama3_df['model'] = 'llama3'
mistral_df['model'] = 'mistral'
phi3_df['model'] = 'phi3'
gemma_df['model'] = 'gemma'
llama3_zero_shot_df['model'] = 'llama3_zero_shot'

# Combine all dataframes into a single dataframe
combined_df = pd.concat([ llama3_df, mistral_df, phi3_df,gemma_df, llama3_zero_shot_df])

# Pivot the dataframe to have models as rows and metrics as columns
pivot_df = combined_df.pivot_table(index='model', columns='metrics', values='baseline')

pivot_df.columns = ["answer_relevancy","context_relevancy",'correctness','faithfulness']

pivot_df = pivot_df[["correctness",'answer_relevancy', 'context_relevancy',"faithfulness"]]

pivot_df = pivot_df.sort_values(by='correctness',ascending=False)

pivot_df

,correctness,answer_relevancy,context_relevancy,faithfulness
model,,,,
llama3,4.182927,0.963415,0.844512,1.0
mistral,4.182927,0.963415,0.832317,1.0
phi3,4.097561,0.963415,0.798780,1.0
gemma,4.085366,0.951220,0.814024,1.0
llama3_zero_shot,3.914634,0.951220,NaN,NaN


In [131]:
sorted_pivot_df.columns = ['answer_relevancy', 'context_relevancy',"correctness","faithfulness"]

In [144]:
pivot_df.to_csv('evaluation.csv', index=True)